In [1]:
import numpy as np
import pandas as pd
import os
import statsmodels
import statsmodels.api as sms
import matplotlib.pyplot as plt
import pathlib
import re
from pathlib import Path
import pickle

from statsmodels.stats.anova import AnovaRM, anova_lm

%matplotlib inline

# Word Length

Strategy:
* Avg10 analysis, then single trials analysis 
* Load all test data results (across the 10 seeds)
* Make empty pandas dataframe, set up headings
* Loop through ica, epoch, scaling, add 
* Then submit to the statistical tests

The decoding results on the test set, during the time windows of interest (450-500 ms; 180-250 ms; 200-400 ms) were all averaged together for each of the 10 runs across all combinations of ICA, baseline correction and feature scaling. 


In [2]:
path = Path('./exp1_confound_corrected/len_matched_results')
folders = os.listdir(path)

In [3]:
# each repetition had a specific seed, so this is the subject id of the ANOVA
seed_map = {'43':0, '54':1, '4':2, '23':3, '56':4, '66':5, '21':6, '9':7, '76':8, '87':9}

In [4]:
# Looking for 450-500 ms average data
# thats time step 112:125

def get_df(n_avg, start_idx, end_idx):
    
    results = []

    for ica in ['strong', 'weak', 'none']:
        for baseline in ['epoch_bc', 'sent_bc', 'no_bc']:
            for scaling in ['mnn', 'ss']:
                foldername = f"ica_{ica}_{baseline}_{scaling}"

                tmp_files = os.listdir(f'{path}/{foldername}')
                tmp_files = [x for x in tmp_files if "test" in x]
                tmp_files = [x for x in tmp_files if f"avg{n_avg}_" in x]
                assert len(tmp_files) > 8

                tmp2 = []
                for i, file in enumerate(tmp_files):
                    seed = re.findall('seed(\d{1,2})', file)[0]
                    if seed in seed_map.keys():
                        tmp = pickle.load(open(path / Path(foldername) / Path(file), 'rb'))
                        #print('loaded file: ', file)
                        tmp_result = np.array(tmp)[start_idx:end_idx].mean()
                        tmp2.append(tmp_result)
                    
                tmp2_mean = np.mean(tmp2)
                results.append([ica, baseline, scaling,  tmp2_mean])
                        
    df = pd.DataFrame(results, columns=['ica', 'baseline','scaling','score'])
    return df

In [5]:
df_avg10 = get_df(n_avg=10, start_idx=112, end_idx=125)
df_avg1 = get_df(n_avg=1, start_idx=112, end_idx=125)

In [6]:
df_avg10

,ica,baseline,scaling,score
0,strong,epoch_bc,mnn,0.526894
1,strong,epoch_bc,ss,0.532532
2,strong,sent_bc,mnn,0.529180
3,strong,sent_bc,ss,0.538199
4,strong,no_bc,mnn,0.550417
5,strong,no_bc,ss,0.554661
6,weak,epoch_bc,mnn,0.546054
7,weak,epoch_bc,ss,0.547504
8,weak,sent_bc,mnn,0.558686
9,weak,sent_bc,ss,0.573145


In [7]:
df_avg1.groupby(['ica','baseline','scaling']).count()

score
ica    baseline scaling       
none   epoch_bc mnn          1
                ss           1
       no_bc    mnn          1
                ss           1
       sent_bc  mnn          1
                ss           1
strong epoch_bc mnn          1
                ss           1
       no_bc    mnn          1
                ss           1
       sent_bc  mnn          1
                ss           1
weak   epoch_bc mnn          1
                ss           1
       no_bc    mnn          1
                ss           1
       sent_bc  mnn          1
                ss           1

# 450-500 ms

In [38]:
df_avg10 = get_df(n_avg=10, start_idx=450//4, end_idx=500//4)
df_avg1 = get_df(n_avg=1, start_idx=450//4, end_idx=500//4)

In [39]:
# Avg 10
model = ols("score ~ C(ica,Sum) + C(baseline,Sum) + C(scaling,Sum) + C(ica,Sum):C(baseline,Sum) + C(ica,Sum):C(scaling,Sum) + C(baseline,Sum):C(scaling,Sum)", data=df_avg10).fit()
aov_table = sms.stats.anova_lm(model, typ=3)
aov_table['result'] = aov_table['PR(>F)'] < 0.01
aov_table['PR(>F)'] = aov_table['PR(>F)'].round(5)
aov_table['F'] = aov_table['F'].round(2)

aov_table

,sum_sq,df,F,PR(>F),result
Intercept,5.366754,1.0,867603.18,0.00000,True
"C(ica, Sum)",0.001840,2.0,148.71,0.00018,True
"C(baseline, Sum)",0.001864,2.0,150.68,0.00017,True
"C(scaling, Sum)",0.000253,1.0,40.94,0.00306,True
"C(ica, Sum):C(baseline, Sum)",0.000232,4.0,9.37,0.02612,False
"C(ica, Sum):C(scaling, Sum)",0.000009,2.0,0.72,0.54231,False
"C(baseline, Sum):C(scaling, Sum)",0.000054,2.0,4.38,0.09819,False
Residual,0.000025,4.0,NaN,NaN,False


In [40]:
# Single trial
model = ols("score ~ C(ica,Sum) + C(baseline,Sum) + C(scaling,Sum) + C(ica,Sum):C(baseline,Sum) + C(ica,Sum):C(scaling,Sum) + C(baseline,Sum):C(scaling,Sum)", data=df_avg1).fit()
aov_table = sms.stats.anova_lm(model, typ=3)
aov_table['result'] = aov_table['PR(>F)'] < 0.01
aov_table['PR(>F)'] = aov_table['PR(>F)'].round(5)
aov_table['F'] = aov_table['F'].round(2)

aov_table

,sum_sq,df,F,PR(>F),result
Intercept,4.659898e+00,1.0,23498462.27,0.00000,True
"C(ica, Sum)",1.191133e-04,2.0,300.33,0.00004,True
"C(baseline, Sum)",5.259648e-05,2.0,132.61,0.00022,True
"C(scaling, Sum)",8.988522e-09,1.0,0.05,0.84182,False
"C(ica, Sum):C(baseline, Sum)",1.085932e-05,4.0,13.69,0.01327,False
"C(ica, Sum):C(scaling, Sum)",3.309060e-08,2.0,0.08,0.92151,False
"C(baseline, Sum):C(scaling, Sum)",3.138174e-07,2.0,0.79,0.51341,False
Residual,7.932261e-07,4.0,NaN,NaN,False


In [42]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df_avg10['score'], groups=df_avg10['baseline'], alpha=0.05)
print('Tukey (baseline)\n\n', m_comp)

m_comp = pairwise_tukeyhsd(endog=df_avg10['score'], groups=df_avg10['ica'], alpha=0.05)
print('Tukey (ICA)\n\n', m_comp)

m_comp = pairwise_tukeyhsd(endog=df_avg10['score'], groups=df_avg10['scaling'], alpha=0.05)
print('Tukey (scaling)\n\n', m_comp)

Tukey (baseline)

 Multiple Comparison of Means - Tukey HSD,FWER=0.05
 group1   group2 meandiff  lower  upper  reject
-----------------------------------------------
epoch_bc  no_bc   0.0249   0.0059 0.0439  True 
epoch_bc sent_bc  0.0114  -0.0077 0.0304 False 
 no_bc   sent_bc -0.0135  -0.0325 0.0055 False 
-----------------------------------------------
Tukey (ICA)

 Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2 meandiff  lower  upper  reject
--------------------------------------------
 none  strong -0.0005  -0.0196 0.0186 False 
 none   weak   0.0212   0.0021 0.0403  True 
strong  weak   0.0217   0.0026 0.0408  True 
--------------------------------------------
Tukey (scaling)

 Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2 meandiff  lower  upper  reject
--------------------------------------------
 mnn     ss    0.0075  -0.0083 0.0233 False 
--------------------------------------------


# 180-220 ms

In [25]:
# between 180-220 ms
df_avg10 = get_df(n_avg=10, start_idx=180//4, end_idx=220//4)
df_avg1 = get_df(n_avg=1, start_idx=180//4, end_idx=220//4)

In [27]:
# avg10
model = ols("score ~ C(ica,Sum) + C(baseline,Sum) + C(scaling,Sum) + C(ica,Sum):C(baseline,Sum) + C(ica,Sum):C(scaling,Sum) + C(baseline,Sum):C(scaling,Sum)", data=df_avg10).fit()
aov_table = sms.stats.anova_lm(model, typ=3)
aov_table['result'] = aov_table['PR(>F)'] < 0.01
aov_table['PR(>F)'] = aov_table['PR(>F)'].round(5)
aov_table['F'] = aov_table['F'].round(2)

aov_table

,sum_sq,df,F,PR(>F),result
Intercept,7.581713e+00,1.0,1315412.45,0.00000,True
"C(ica, Sum)",1.844272e-03,2.0,159.99,0.00015,True
"C(baseline, Sum)",1.293955e-03,2.0,112.25,0.00031,True
"C(scaling, Sum)",3.251566e-09,1.0,0.00,0.98219,False
"C(ica, Sum):C(baseline, Sum)",6.210289e-04,4.0,26.94,0.00375,True
"C(ica, Sum):C(scaling, Sum)",3.956180e-05,2.0,3.43,0.13557,False
"C(baseline, Sum):C(scaling, Sum)",1.381969e-05,2.0,1.20,0.39091,False
Residual,2.305501e-05,4.0,NaN,NaN,False


In [36]:
df_avg10['combination'] = df_avg10.ica + " / " + df_avg10.baseline

m_comp = pairwise_tukeyhsd(endog=df_avg10['score'], groups=df_avg10['combination'], alpha=0.01)
print('Tukey (ica-baseline)\n\n', m_comp)

Tukey (ica-baseline)

          Multiple Comparison of Means - Tukey HSD,FWER=0.01        
      group1            group2      meandiff  lower   upper  reject
-------------------------------------------------------------------
 none / epoch_bc     none / no_bc    0.0111   -0.004  0.0262 False 
 none / epoch_bc    none / sent_bc  -0.0157  -0.0308 -0.0006  True 
 none / epoch_bc  strong / epoch_bc -0.0207  -0.0358 -0.0056  True 
 none / epoch_bc    strong / no_bc  -0.0072  -0.0223  0.0079 False 
 none / epoch_bc   strong / sent_bc -0.0326  -0.0477 -0.0175  True 
 none / epoch_bc   weak / epoch_bc   -0.038  -0.0531 -0.0229  True 
 none / epoch_bc     weak / no_bc   -0.0152  -0.0303 -0.0001  True 
 none / epoch_bc    weak / sent_bc  -0.0218  -0.0369 -0.0067  True 
   none / no_bc     none / sent_bc  -0.0268  -0.0419 -0.0117  True 
   none / no_bc   strong / epoch_bc -0.0318  -0.0469 -0.0167  True 
   none / no_bc     strong / no_bc  -0.0184  -0.0335 -0.0033  True 
   none / no_bc    strong

In [ ]:
m_comp = pairwise_tukeyhsd(endog=df_avg10['score'], groups=df_avg10['baseline'], alpha=0.05)
print('Tukey (baseline)\n\n', m_comp)

In [34]:
# single trial
model = ols("score ~ C(ica,Sum) + C(baseline,Sum) + C(scaling,Sum) + C(ica,Sum):C(baseline,Sum) + C(ica,Sum):C(scaling,Sum) + C(baseline,Sum):C(scaling,Sum)", data=df_avg1).fit()
aov_table = sms.stats.anova_lm(model, typ=3)
aov_table['result'] = aov_table['PR(>F)'] < 0.01
aov_table['PR(>F)'] = aov_table['PR(>F)'].round(5)
aov_table['F'] = aov_table['F'].round(2)

aov_table

,sum_sq,df,F,PR(>F),result
Intercept,5.115141e+00,1.0,6555156.87,0.00000,True
"C(ica, Sum)",4.516641e-04,2.0,289.41,0.00005,True
"C(baseline, Sum)",5.833986e-05,2.0,37.38,0.00258,True
"C(scaling, Sum)",5.530311e-05,1.0,70.87,0.00109,True
"C(ica, Sum):C(baseline, Sum)",1.978070e-05,4.0,6.34,0.05066,False
"C(ica, Sum):C(scaling, Sum)",1.163600e-05,2.0,7.46,0.04474,False
"C(baseline, Sum):C(scaling, Sum)",8.933929e-07,2.0,0.57,0.60446,False
Residual,3.121293e-06,4.0,NaN,NaN,False


In [35]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df_avg1['score'], groups=df_avg1['baseline'], alpha=0.01)
print('Tukey (baseline)\n\n', m_comp)

m_comp = pairwise_tukeyhsd(endog=df_avg1['score'], groups=df_avg1['ica'], alpha=0.01)
print('Tukey (ICA)\n\n', m_comp)

m_comp = pairwise_tukeyhsd(endog=df_avg1['score'], groups=df_avg1['scaling'], alpha=0.01)
print('Tukey (scaling)\n\n', m_comp)

Tukey (baseline)

 Multiple Comparison of Means - Tukey HSD,FWER=0.01
 group1   group2 meandiff  lower  upper  reject
-----------------------------------------------
epoch_bc  no_bc   0.0022  -0.0097 0.0141 False 
epoch_bc sent_bc -0.0022  -0.0141 0.0096 False 
 no_bc   sent_bc -0.0044  -0.0163 0.0075 False 
-----------------------------------------------
Tukey (ICA)

 Multiple Comparison of Means - Tukey HSD,FWER=0.01
group1 group2 meandiff  lower  upper  reject
--------------------------------------------
 none  strong -0.0122  -0.0184 -0.006  True 
 none   weak  -0.0052  -0.0114 0.001  False 
strong  weak   0.007    0.0008 0.0132  True 
--------------------------------------------
Tukey (scaling)

 Multiple Comparison of Means - Tukey HSD,FWER=0.01
group1 group2 meandiff  lower  upper  reject
--------------------------------------------
 mnn     ss    0.0035  -0.0045 0.0115 False 
--------------------------------------------


# 200-400 ms

In [43]:
df_avg10 = get_df(n_avg=10, start_idx=200//4, end_idx=400//4)
df_avg1 = get_df(n_avg=1, start_idx=200//4, end_idx=400//4)

# avg10
model = ols("score ~ C(ica,Sum) + C(baseline,Sum) + C(scaling,Sum) + C(ica,Sum):C(baseline,Sum) + C(ica,Sum):C(scaling,Sum) + C(baseline,Sum):C(scaling,Sum)", data=df_avg10).fit()
aov_table_avg10 = sms.stats.anova_lm(model, typ=3)
aov_table_avg10['result'] = aov_table_avg10['PR(>F)'] < 0.01
aov_table_avg10['PR(>F)'] = aov_table_avg10['PR(>F)'].round(5)
aov_table_avg10['F'] = aov_table_avg10['F'].round(2)

# avg1
model = ols("score ~ C(ica,Sum) + C(baseline,Sum) + C(scaling,Sum) + C(ica,Sum):C(baseline,Sum) + C(ica,Sum):C(scaling,Sum) + C(baseline,Sum):C(scaling,Sum)", data=df_avg1).fit()
aov_table_avg1 = sms.stats.anova_lm(model, typ=3)
aov_table_avg1['result'] = aov_table_avg1['PR(>F)'] < 0.01
aov_table_avg1['PR(>F)'] = aov_table_avg1['PR(>F)'].round(5)
aov_table_avg1['F'] = aov_table_avg1['F'].round(2)

In [44]:
aov_table_avg10

,sum_sq,df,F,PR(>F),result
Intercept,6.341948,1.0,1118730.04,0.00000,True
"C(ica, Sum)",0.000549,2.0,48.42,0.00157,True
"C(baseline, Sum)",0.000797,2.0,70.28,0.00077,True
"C(scaling, Sum)",0.000336,1.0,59.19,0.00154,True
"C(ica, Sum):C(baseline, Sum)",0.000586,4.0,25.84,0.00406,True
"C(ica, Sum):C(scaling, Sum)",0.000007,2.0,0.60,0.59366,False
"C(baseline, Sum):C(scaling, Sum)",0.000011,2.0,0.96,0.45684,False
Residual,0.000023,4.0,NaN,NaN,False


In [45]:
aov_table_avg1

,sum_sq,df,F,PR(>F),result
Intercept,4.846238e+00,1.0,26291255.75,0.00000,True
"C(ica, Sum)",2.065008e-04,2.0,560.14,0.00001,True
"C(baseline, Sum)",2.473497e-05,2.0,67.09,0.00084,True
"C(scaling, Sum)",1.093929e-05,1.0,59.35,0.00153,True
"C(ica, Sum):C(baseline, Sum)",1.112927e-05,4.0,15.09,0.01110,False
"C(ica, Sum):C(scaling, Sum)",9.161128e-07,2.0,2.48,0.19885,False
"C(baseline, Sum):C(scaling, Sum)",2.326048e-07,2.0,0.63,0.57788,False
Residual,7.373155e-07,4.0,NaN,NaN,False


In [48]:
# avg10 baseline-ica interaction

df_avg10['combination'] = df_avg10.ica + " / " + df_avg10.baseline
m_comp = pairwise_tukeyhsd(endog=df_avg10['score'], groups=df_avg10['combination'], alpha=0.05)
print('Tukey (ica-baseline)\n\n', m_comp)

Tukey (ica-baseline)

          Multiple Comparison of Means - Tukey HSD,FWER=0.05        
      group1            group2      meandiff  lower   upper  reject
-------------------------------------------------------------------
 none / epoch_bc     none / no_bc    0.0089  -0.0167  0.0345 False 
 none / epoch_bc    none / sent_bc  -0.0132  -0.0388  0.0123 False 
 none / epoch_bc  strong / epoch_bc -0.0158  -0.0414  0.0098 False 
 none / epoch_bc    strong / no_bc  -0.0055   -0.031  0.0201 False 
 none / epoch_bc   strong / sent_bc -0.0231  -0.0487  0.0025 False 
 none / epoch_bc   weak / epoch_bc  -0.0199  -0.0454  0.0057 False 
 none / epoch_bc     weak / no_bc    0.0024  -0.0232  0.0279 False 
 none / epoch_bc    weak / sent_bc  -0.0011  -0.0266  0.0245 False 
   none / no_bc     none / sent_bc  -0.0221  -0.0477  0.0034 False 
   none / no_bc   strong / epoch_bc -0.0247  -0.0503  0.0009 False 
   none / no_bc     strong / no_bc  -0.0144  -0.0399  0.0112 False 
   none / no_bc    strong

In [49]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df_avg1['score'], groups=df_avg1['baseline'], alpha=0.01)
print('Tukey (baseline)\n\n', m_comp)

m_comp = pairwise_tukeyhsd(endog=df_avg1['score'], groups=df_avg1['ica'], alpha=0.01)
print('Tukey (ICA)\n\n', m_comp)

m_comp = pairwise_tukeyhsd(endog=df_avg1['score'], groups=df_avg1['scaling'], alpha=0.01)
print('Tukey (scaling)\n\n', m_comp)

Tukey (baseline)

 Multiple Comparison of Means - Tukey HSD,FWER=0.01
 group1   group2 meandiff  lower  upper  reject
-----------------------------------------------
epoch_bc  no_bc   0.0016  -0.0061 0.0094 False 
epoch_bc sent_bc -0.0012   -0.009 0.0065 False 
 no_bc   sent_bc -0.0029  -0.0106 0.0049 False 
-----------------------------------------------
Tukey (ICA)

 Multiple Comparison of Means - Tukey HSD,FWER=0.01
group1 group2 meandiff  lower   upper  reject
---------------------------------------------
 none  strong -0.0082  -0.0117 -0.0046  True 
 none   weak  -0.0028  -0.0064  0.0007 False 
strong  weak   0.0053   0.0018  0.0089  True 
---------------------------------------------
Tukey (scaling)

 Multiple Comparison of Means - Tukey HSD,FWER=0.01
group1 group2 meandiff  lower  upper  reject
--------------------------------------------
 mnn     ss    0.0016  -0.0038 0.0069 False 
--------------------------------------------
